# Advanced Lane Finding
In order to manipulate a car I need to measure how much my lane is curving. To do that I need to map out the lanes in my camera image.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

def arrange_side_by_side(img1, img2, title1, title2):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 30))
    ax1.imshow(img1)
    ax1.set_title(title1)
    ax2.imshow(img2, cmap='gray')
    ax2.set_title(title2)
    plt.show()

## Compute the Camera Calibration and Distortion Coefficients
Camera image has distortions. This means camera image does not represent perfectly real world objects. There is two types of distortion, Radial Distortion and Tangental Distortion.

To get a curvature of lane lines based on real world I need to correct those distortions, before I map out the lane lines from a camera image,.

To correct distortions I need to calculate some parameters.
 - distortion coefficients (k1, k2, p1, p2, k3)
 - camera matrix ([[fx, 0, xc], [0, fy, yc], [0, 0, 1]]) <= intrinsic parameters
 - extrinsic parameters (rotation and translation vector)
 
To find those parameters I'll do calibration through chessboard images.
 1. detect image points on 2D chessboard image space using findChessboardCorners() function
 2. prepare object points on 3D real world space which indicate same points with 1st step.
 3. get parameters for correcting image distortion using calibrateCamera() function.
 4. finally correct distortions using undistort() function.

In [ ]:
fname = 'camera_cal/calibration2.jpg'
img = mpimg.imread(fname)

img_origin = np.copy(img)
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

plt.imshow(img_gray, cmap='gray')

In [ ]:
# 1st step: find image points on 2D chessboard image
pattern_size = (9, 6)
found, image_points_base = cv2.findChessboardCorners(img_gray, pattern_size, None)

# checking
cv2.drawChessboardCorners(img, pattern_size, image_points_base, found)
arrange_side_by_side(img_origin, img, 'original image', 'draw corners')

In [ ]:
# 2nd step: prepare object points
object_points_base = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
object_points_base[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape((-1, 2))

# x = np.arange(pattern_size[0])
# y = np.arange(pattern_size[1])
# xx, yy = np.meshgrid(x,y)
# object_points_base = [[xx[i][j], yy[i][j], 0] for j in range(pattern_size[0]) for i in range(pattern_size[1])]

object_points = []
image_points = []

if found:
    object_points.append(object_points_base)
    image_points.append(image_points_base)

In [ ]:
# 3rd step: calibrate camera to get parameters for undistortion
try:
    r, cam_mtx, dist_coeff, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, img.shape[1::-1], None, None)
except:
    print('Exception occured at calibrateCamera function!!\nPlease check the parameters.')

In [ ]:
# 4th step: Undistortion
undist_img = cv2.undistort(img_origin, cam_mtx, dist_coeff)

In [ ]:
# See the result
arrange_side_by_side(img_origin, undist_img, 'original', 'undistorted')

## Create Thresholded Binary Image

In [3]:
def filter_sobel(img, orientation='x'):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orientation == 'x':
        sobel = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0)
    elif orientation == 'y':
        sobel = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1)
    else:
        print("Unknown value of orientation.")
        return None
    abs_img = np.absolute(sobel)
    return abs_img * 255 / np.max(abs_img)

def filter_sobel_gradient_magnitude(img, ksize=3):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=ksize)
    grad_mag = np.sqrt(sobelx**2 + sobely**2)
    return grad_mag * 255 / np.max(grad_mag)

def filter_gradient_direction(img, ksize=3):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    abs_sobelx = np.absolute(cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=ksize))
    abs_sobely = np.absolute(cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=ksize))
    return np.arctan2(abs_sobely, abs_sobelx)
    
def filter_hls(rgb_img, channel='s'):
    if channel == 'h':
        c = 0
    elif channel == 'l':
        c = 1
    elif channel == 's':
        c = 2
    else:
        print("Undefined parameter channel")
    hls = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HLS)
    return hls[:,:,c]

def create_binary_image(img, thresh_min=50, thresh_max=100):
    binary = np.zeros_like(img)
    binary[(img > thresh_min) & (img < thresh_max)] = 1
    return binary

In [ ]:
# apply the Sobel filter for y direction
#sobelx = filter_sobel(rimg, 'x')
#binary_sobelx = create_binary_image(sobelx, 15, 140)
#
#arrange_side_by_side(rimg, binary_sobelx, 'Origin', 'BinaryX')

In [ ]:
# apply the Sobel filter for y direction
#sobely = filter_sobel(rimg, 'y')
#binary_sobely = create_binary_image(sobely, 20, 150)
#
#arrange_side_by_side(rimg, binary_sobely, 'Origin', 'Binary')

In [ ]:
grad_mag = filter_sobel_gradient_magnitude(rimg, ksize=5)
binary_grad_mag = create_binary_image(grad_mag, 30, 170)

arrange_side_by_side(rimg, binary_grad_mag, 'Origin', 'Gradient Magnitude Thresh')

In [ ]:
grad_dir = filter_gradient_direction(rimg, ksize=11)
binary_grad_dir = create_binary_image(grad_dir, 0.8, 1.3)

arrange_side_by_side(rimg, binary_grad_dir, 'Origin', 'Gradient Direction')

In [ ]:
s_img = filter_hls(rimg, 's')
binary_satulate = create_binary_image(s_img, 150, 300)
arrange_side_by_side(rimg, binary_satulate, 'Origin', 'Satulate Channel')

In [ ]:
# Combine the above binary images
binary = np.zeros_like(binary_grad_dir)
binary[((binary_grad_mag == 1) & (binary_grad_dir == 1) | (binary_satulate == 1))] = 1

arrange_side_by_side(rimg, binary, 'Origin', 'Combined Binary Image')

## Apply Perspective Transform

In [ ]:
#%matplotlib qt
%matplotlib inline

src = np.float32(
    [[840, 546],
     [1051, 677],
     [244, 686],
     [453, 546]])
dst = np.float32(
    [[1051, 463],
     [1051, 686],
     [245, 686],
     [245, 463]])

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

In [ ]:
warped = cv2.warpPerspective(binary, M, (rimg.shape[1], rimg.shape[0]), flags=cv2.INTER_LINEAR)

arrange_side_by_side(rimg, warped, 'Original', 'Bird Eyes View')

## Sliding Windows to plot lines

In [4]:
def get_starting_point_likely(img):
    hist = np.sum(img[img.shape[0]//2:, :], axis=0)
    center_horizontal = img.shape[1]//2
    left = np.argmax(hist[:center_horizontal])
    right = np.argmax(hist[center_horizontal:]) + center_horizontal
    return left, right

def search_lane_line_pixels(img, left_start, right_start, draw_rect=False):
    margin = 100
    n_window = 9
    minpix_thresh = 50
    
    nonzero = img.nonzero()
    nonzero_x = np.array(nonzero[1])
    nonzero_y = np.array(nonzero[0])

    # output place holders
    left_lane_pixels = []
    right_lane_pixels = []
    out_img = np.dstack((img, img, img)) * 255
    
    for i in range(n_window):
        bottom = img.shape[0] - img.shape[0]//n_window * i
        top = bottom - img.shape[0]//n_window
        l_left = left_start - margin
        l_right = left_start + margin
        r_left = right_start - margin
        r_right = right_start + margin
        
        if draw_rect:
            cv2.rectangle(out_img, (l_left, top), (l_right, bottom), (0,255,0), 2)
            cv2.rectangle(out_img, (r_left, top), (r_right, bottom), (0,255,0), 2)
        
        left_inds = ((nonzero_y > top) & (nonzero_y < bottom) & (nonzero_x > l_left) & (nonzero_x < l_right)).nonzero()[0]
        right_inds = ((nonzero_y > top) & (nonzero_y < bottom) & (nonzero_x > r_left) & (nonzero_x < r_right)).nonzero()[0]
        left_lane_pixels.append(left_inds)
        right_lane_pixels.append(right_inds)
        
        if len(left_inds > minpix_thresh):
            left_start = np.int(np.mean(nonzero_x[left_inds]))
        if len(right_inds > minpix_thresh):
            right_start = np.int(np.mean(nonzero_x[right_inds]))
            
    try:
        left_lane_pixels = np.concatenate(left_lane_pixels)
        right_lane_pixels = np.concatenate(right_lane_pixels)
    except ValueError:
        pass
    
    lx = nonzero_x[left_lane_pixels]
    ly = nonzero_y[left_lane_pixels]
    rx = nonzero_x[right_lane_pixels]
    ry = nonzero_y[right_lane_pixels]
    
    out_img[ly, lx] = [255, 0, 0]
    out_img[ry, rx] = [0, 0, 255]
    
    return lx, ly, rx, ry ,out_img

In [5]:
def get_quadratic_plots(img, quad_coef):
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    try:
        plotx = quad_coef[0]*ploty**2 + quad_coef[1]*ploty + quad_coef[2]
    except TypeError:
        plotx = 1*ploty**2 + 1*ploty
    return ploty, plotx

In [ ]:
# Blind search execution
starting_point_l, starting_point_r = get_starting_point_likely(warped)
lx, ly, rx, ry, rect_drawed = search_lane_line_pixels(warped, starting_point_l, starting_point_r, True)

coeff_left = np.polyfit(ly, lx, 2)
coeff_right = np.polyfit(ry, rx, 2)
ploty, plotx_l = get_quadratic_plots(warped, coeff_left)
ploty, plotx_r = get_quadratic_plots(warped, coeff_right)

plt.imshow(rect_drawed)
plt.plot(plotx_l, ploty, color='yellow')
plt.plot(plotx_r, ploty, color='yellow')
plt.xlim(0, rect_drawed.shape[1])
plt.ylim(rect_drawed.shape[0], 0)

## Use the previous polynomial to skip the sliding window
So far I did the blind search. However using the full alforithm from before and starting fresh on every frame may seem inefficient, as the lane lines don't necessarily move a lot from frame to frame.

Instead I can just search in a margin around the previous lane line position. So, once I know where the lines are in one frame of video I can do a highly targeted search for them in the next frame.

And this way should help me track the lanes through sharp curves and tricky conditions. If I lose track of the lines, go back to my sliding windos search or other method to rediscover them.

To hold the pixel values contained within the boundaries of a given sliding window I used the variables called <code>left_lane_pixels</code> and <code>right_lane_pixels</code>. This time, we'll take the polynomial functions I fit before, along with a hyperparameter <code>margin</code>, to determine whivh activated pixcels fall into the green shaded areas from the avobe image.

To implement this, I'll want to grab only those pixels with x values that are +/- my margin from my polynomial lines.

In [6]:
def get_search_area_idx(prev_binary, prev_coefs, margin=100):
    idx_nonzero = prev_binary.nonzero()
    idx_nonzero_y = np.array(idx_nonzero[0])
    idx_nonzero_x = np.array(idx_nonzero[1])
    prev_polynomial = prev_coefs[0] * idx_nonzero_y**2 + prev_coefs[1] * idx_nonzero_y + prev_coefs[2]
    is_target = (prev_polynomial-margin < idx_nonzero_x) & (prev_polynomial+margin > idx_nonzero_x)
    idx_nonzero_y = idx_nonzero_y[is_target]
    idx_nonzero_x = idx_nonzero_x[is_target]
    return idx_nonzero_y, idx_nonzero_x


In [7]:
def get_quadratic_coefficient(binary_image, ym_per_pix, xm_per_pix):
    nonzero = np.nonzero(binary_image)
    nonzero_y = nonzero[0]
    nonzero_x = nonzero[1]
    return np.polyfit(nonzero_y* ym_per_pix, nonzero_x*xm_per_pix, 2)

def search_polynomial_around_prior(new_binary_image, target_y, target_x, ym_per_pix=1, xm_per_pix=1):
    new_image = np.zeros_like(new_binary_image)
    new_image[target_y, target_x] = 1
    coefficient = get_quadratic_coefficient(new_image, ym_per_pix, xm_per_pix)
    return coefficient

In [ ]:
target_ly, target_lx = get_search_area_idx(warped, coeff_left, 80)
target_ry, target_rx = get_search_area_idx(warped, coeff_right, 80)

In [ ]:
new_image = np.copy(warped)
coeff_left = search_polynomial_around_prior(new_image, target_ly, target_lx)
coeff_right = search_polynomial_around_prior(new_image, target_ry, target_rx)

In [ ]:
ploty, plotx_l = get_quadratic_plots(warped, coeff_left)
ploty, plotx_r = get_quadratic_plots(warped, coeff_right)

plt.imshow(new_image, cmap='gray')
plt.plot(plotx_l, ploty, color='red')
plt.plot(plotx_r, ploty, color='red')
plt.show()

## Measuring Curvature
Next I'll compute the radius of curvature of a polinomial. I'm fitting for f(y), rather than f(x), because the lane lines in the warped image are near vartical and may have the same x value for more than one y value.

### Radius of Curvature
The radius of curvature at any point x of the function x=f(y) is given as follows:

    f(y) = Ay**2 + By + C = x
    Rcurve = [1+(dx/dy)**2]**3/2 / |d**2x/dy**2|
    
In the case of the second order polynomial above, the first and second derivatives are:

    f'(y) = dx/dy = 2Ay + B
    f"(y) = d**2x/dy**2 = 2A
   
So, our equation for radius of curvature becomes:
    
    Rcurve = (1+(2Ay+B)**2)**3/2 / |2A|
    
The y values of my image increase from top to bottom, so if, for example, I wanted to measure the radius of curvature closest to my vehicle, I could evaluate the formula above at the y value corresponding to the bottom of my image, or in Python, at yvalue = image.shape[0].

In [8]:
def measure_curvature(coeff, ploty, ymeters_per_pix=1):
    y_eval = np.max(ploty)*ymeters_per_pix
    d1_poly = 2*coeff[0]*y_eval + coeff[1]
    d2_poly = 2*coeff[0]
    return (1+(d1_poly**2))**1.5 / np.absolute(d2_poly)

In [ ]:
left_curve = measure_curvature(coeff_left, ploty)
right_curve = measure_curvature(coeff_right, ploty)

## Measureing Physical Lane Curvature
#### Measureing How long and wide the section of lane
We could do this in detail by measuring out the physical lane in the field of view of the camera, but for this project, I can assume that if I'm projecting a section of lane similar to the images above, the lane is about 30 meaters long and 3.7 meters wide.

Let's say that our camera image has 720 relevant pixels in the y-dimension (remember, our image is perspective-transformed), and we'll say roughly 700 relevant pixels in the x-dimension (if from 200 pixels on the left to 900 on the right, or 700). Therefore, to convert from pixels to real-world meter measurements, I can use(in this case):

    ym_per_pix = 30/720
    xm_per_pix = 3.7/700

In [ ]:
ym_per_pix = 30/warped.shape[0]
xm_per_pix = 3.7/ (starting_point_r - starting_point_l)
real_coeff_left = search_polynomial_around_prior(new_image, target_ly, target_lx, ym_per_pix, xm_per_pix)
real_coeff_right = search_polynomial_around_prior(new_image, target_ry, target_rx, ym_per_pix, xm_per_pix)

In [ ]:
left_curve = measure_curvature(real_coeff_left, ploty, ym_per_pix)
right_curve = measure_curvature(real_coeff_right,  ploty, ym_per_pix)

In [ ]:
print(left_curve, 'm', right_curve, 'm')

#### Vehicle position with respect to center of the road

In [9]:
def get_xvalues_on_polynomial_line(coefficient, ploty):
    return coefficient[0]*ploty**2 + coefficient[1]*ploty + coefficient[2]

def measure_difference_with_center_road(image_width, plotx_left, plotx_right):
    for i in range(len(plotx_left)):
        left_position = plotx_left[-i-1]
        right_position = plotx_right[-i-1]
        if (left_position > 0 and left_position < image_width) and\
            (right_position > 0 and right_position < image_width):
            difference = (image_width - right_position) - left_position
            break
    return difference

In [ ]:
plotx_left = get_xvalues_on_polynomial_line(coeff_left, ploty)
plotx_right = get_xvalues_on_polynomial_line(coeff_right, ploty)

difference = measure_difference_with_center_road(warped.shape[1], plotx_left, plotx_right)
if difference > 0:
    diff_result = "Vehicle is {}m right of center".format(np.round(difference*xm_per_pix, 2))
elif difference < 0:
    diff_result = "Vehicle is {}m left of center".format(np.round(-difference*xm_per_pix, 2))
else:
    diff_result = "Vehicle is Just Center!!"

In [10]:
def draw_lane_area(image, plotx_left, plotx_right, color=[0,255,0]):
    canvas = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    mgrid = np.mgrid[0:image.shape[0], 0:image.shape[1]]
    is_lane_area =[]
    for i, row in enumerate(mgrid[1]):
        is_lane = (row > plotx_left[i]) & (row < plotx_right[i])
        is_lane_area.append(is_lane)
    canvas[is_lane_area] = color
    return canvas

In [11]:
def color_lane_lines(binary_img, ly, lx, ry, yx, l_color=[255,0,0], r_color=[0,0,255]):
    out_img = np.dstack((binary_img, binary_img, binary_img))
    out_img[ly, lx] = l_color
    out_img[ry, yx] = r_color
    return out_img.astype('uint8')

In [ ]:
area_colored_img = draw_lane_area(warped, plotx_left, plotx_right)
line_colored_img = color_lane_lines(warped, target_ly, target_lx, target_ry, target_rx)
combined = area_colored_img + line_colored_img
unwarped = cv2.warpPerspective(combined, Minv, (rimg.shape[1], rimg.shape[0]), flags=cv2.INTER_LINEAR)

In [ ]:
plt.imshow(unwarped)

In [12]:
def draw_mask(initial_img, mask_img, α=0.8, β=0.8, γ=0.):
    return cv2.addWeighted(initial_img, α, mask_img, β, γ)

In [ ]:
masked_lane_area = draw_mask(unwarped, rimg)

In [ ]:
text1 = "Radius of Curvature = {}(m)".format(np.round(left_curve, -1))
text2 = "Vehicle is {}".format(diff_result)
masked_lane_area = cv2.putText(masked_lane_area, text1, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 5)
masked_lane_area = cv2.putText(masked_lane_area, text2, (30, 140), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 5)

In [ ]:
plt.imshow(masked_lane_area)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_video))

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML

from calibrate import Calibration
from input_holder import InputHolder

In [35]:
def to_binary(image):
    grad_mag = filter_sobel_gradient_magnitude(image, ksize=5)
    grad_dir = filter_gradient_direction(image, ksize=11)
    saturate = filter_hls(image, 's')
    
    binary_grad_mag = create_binary_image(grad_mag, 30, 170)
    binary_grad_dir = create_binary_image(grad_dir, 0.8, 1.3)
    binary_satulate = create_binary_image(saturate, 150, 300)
    
    binary = np.zeros_like(binary_grad_dir)
    binary[((binary_grad_mag == 1) & (binary_grad_dir == 1) | (binary_satulate == 1))] = 1
    return binary
    
def normalize_coefficients(current_coeffs, prev_coeffs, prev_weight=0.0):
    norm_a = prev_coeffs[0]*prev_weight + current_coeffs[0]*(1-prev_weight)   
    norm_b = prev_coeffs[1]*prev_weight + current_coeffs[1]*(1-prev_weight)  
    return (norm_a, norm_b)

def process_image(image, holder):
    global is_first_frame
    global prev_coeff_left
    global prev_coeff_right
    
    undist_img = cv2.undistort(image, holder.calib.cam_mtx, holder.calib.dist_coeff)
    binary_img = to_binary(undist_img)
    warped_img = cv2.warpPerspective(binary_img, holder.M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    
    print(is_first_frame)
    if is_first_frame:
        starting_point_l, starting_point_r = get_starting_point_likely(warped_img)        
        lx_pixels, ly_pixels, rx_pixels, ry_pixels, lane_colored_img = search_lane_line_pixels(warped_img, starting_point_l, starting_point_r)
        coeff_left = np.polyfit(ly_pixels, lx_pixels, 2)
        coeff_right = np.polyfit(ry_pixels, rx_pixels, 2)
        is_first_frame = False
    else:
        ly_pixels, lx_pixels = get_search_area_idx(warped_img, prev_coeff_left, 80)
        ry_pixels, rx_pixels = get_search_area_idx(warped_img, prev_coeff_right, 80)
        coeff_left = search_polynomial_around_prior(image, ly_pixels, lx_pixels)
        coeff_right = search_polynomial_around_prior(image, ry_pixels, rx_pixels)
    
    coeff_left = normalize_coefficients(coeff_left, prev_coeff_left, 0.9)
    coeff_right = normalize_coefficients(coeff_right, prev_coeff_right, .9)
    
    ploty, plotx_l = get_quadratic_plots(warped_img, coeff_left)
    ploty, plotx_r = get_quadratic_plots(warped_img, coeff_right)
    left_curve = measure_curvature(coeff_left, ploty)
    right_curve = measure_curvature(coeff_right, ploty)
    
    # draw lane mask (lane area & lane line)
    area_colored_img = draw_lane_area(warped_img, plotx_l, plotx_r)
    line_colored_img = color_lane_lines(warped_img, target_ly, target_lx, target_ry, target_rx)
    combined = area_colored_img + line_colored_img
    unwarped = cv2.warpPerspective(combined, holder.Minv, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    masked_lane_area = draw_mask(unwarped, image)
    
    # get curvature
    ym_per_pix = real_ym/warped_img.shape[0]
    xm_per_pix = real_xm/ (starting_point_r - starting_point_l)
    real_coeff_left = search_polynomial_around_prior(image, target_ly, target_lx, ym_per_pix, xm_per_pix)
    real_coeff_right = search_polynomial_around_prior(image, target_ry, target_rx, ym_per_pix, xm_per_pix)
    real_left_curve = measure_curvature(real_coeff_left, ploty, ym_per_pix)
    real_right_curve = measure_curvature(real_coeff_right,  ploty, ym_per_pix)
    
    # get difference from center
    difference = measure_difference_with_center_road(warped_img.shape[1], plotx_l, plotx_r)
    if difference > 0:
        diff_result = "Vehicle is {}m right of center".format(np.round(difference*xm_per_pix, 2))
    elif difference < 0:
        diff_result = "Vehicle is {}m left of center".format(np.round(-difference*xm_per_pix, 2))
    else:
        diff_result = "Vehicle is Just Center!!"
    text1 = "Radius of Curvature = {}(m)".format(np.round(left_curve, -1))
    text2 = "Vehicle is {}".format(diff_result)
    masked_lane_area = cv2.putText(masked_lane_area, text1, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 5)
    masked_lane_area = cv2.putText(masked_lane_area, text2, (30, 140), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 5)    
    
    prev_coeff_left = coeff_left
    prev_coeff_right = coeff_right
    
    return masked_lane_area

In [32]:
# camera calibration
fnames = glob.glob('camera_cal/*.jpg')
pattern_size = (9, 6)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
calib = Calibration(fnames, pattern_size, criteria)

In [36]:
# Static values
src = np.float32(
    [[840, 546],
     [1051, 677],
     [244, 686],
     [453, 546]])
dst = np.float32(
    [[1051, 463],
     [1051, 686],
     [245, 686],
     [245, 463]])
holder = InputHolder(calib, src, dst)

In [37]:
# Changable values
is_first_frame = True
prev_coeff_left
prev_coeff_right

NameError: name 'prev_coeff_left' is not defined

In [34]:
test_video = 'output_images/lane_drawed_project_video.mp4'
clip = VideoFileClip('project_video.mp4')
binary_clip = clip.fl_image(lambda image: process_image(image, holder))
%time binary_clip.write_videofile(test_video, audio=False)

True


NameError: name 'prev_coeff_left' is not defined